## Indicadores de Salud de la Ciudad de México

In [1]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, readxl, janitor, stringr, writexl, foreign, downloader, tools)

Cargando paquete requerido: pacman



Descargar datos de Defunciones Regsitradas de INEGI, que es la que se toma de acuerdo con Secretaría de Salud

In [2]:
#Definiciones para antes de descargar los datos
# Crear directorio principal
dir_datos <- 'Estadísticas de Defunciones Registradas (EDR)/Microdatos_1'
if (!dir.exists(dir_datos)) {
  dir.create(dir_datos, recursive = TRUE)
}
print(file.exists(dir_datos)) #True = directorio creado

[1] TRUE


In [4]:
# Función para descargar y procesar los dbf de defunciones
descargar_defunciones_dbf <- function() {
  options(timeout = 90000)
  lista_defun <- list()
  
  # Lista de URLs con sus años o periodos
  urls <- list(
    '2023' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2023/defunciones_base_datos_2023_dbf.zip',
    '2022' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2022/defunciones_base_datos_2022_dbf.zip',
    '2021' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2021/defunciones_base_datos_2021_dbf.zip',
    '2020' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2020/defunciones_base_datos_2020_dbf.zip'
    #'2015_2019' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/datos/defunciones_generales_base_datos_2015_2019_dbf.zip'
  )
  
  for (periodo in names(urls)) {
    url <- urls[[periodo]]
    zip_path <- file.path(dir_datos, basename(url))
    download.file(url, zip_path, mode = "wb")
    
    # Carpeta para extraer
    unzip_dir <- file.path(dir_datos, paste0('Defunciones_', periodo))
    dir.create(unzip_dir, showWarnings = FALSE)
    
    unzip(zip_path, exdir = unzip_dir)
    unlink(zip_path)  # borrar zip
    
    # Buscar archivos .dbf en el unzip_dir
    dbf_files <- list.files(unzip_dir, pattern = '\\.dbf$', full.names = TRUE, ignore.case = TRUE)
    
    if(length(dbf_files) == 0) {
      warning(paste0('No se encontró archivo DBF para ', periodo))
      next
    }
    
    # Leer cada DBF y guardarlo en lista
    period_lista_defun <- list()
    for(dbf_file in dbf_files) {
      nombre <- file_path_sans_ext(basename(dbf_file))
      df <- tryCatch({
        read.dbf(dbf_file, as.is = TRUE) %>% as_tibble() %>% 
        clean_names()
      }, error = function(e) {
        warning(paste0('Error leyendo DBF: ', dbf_file))
        NULL
      })
      
      if(!is.null(df)) {
        period_lista_defun[[nombre]] <- df
      }
    }
    
    lista_defun[[periodo]] <- period_lista_defun
  }
  
  save(lista_defun, file = file.path(dir_datos, 'defunciones_lista_defun.RData'))
  return(lista_defun)
}

In [5]:
# Ejecutar descarga
defunciones <- descargar_defunciones_dbf()

In [6]:
#Guardar la lista completa para no volver a descargar los datos
saveRDS(defunciones, file = 'Estadísticas de Defunciones Registradas (EDR)/edr_data.rds')

Descargar los datos del Subsistema de Información sobre Nacimientos (SINAC) de la Secretaría de Salud Federal.

*Comparando los nacimientos de la tabla de nacimientos de datos abiertos, con los de los tabulados de Secretaría de Salud, vemos que los datos ya están ajustados con CONAPO*

In [13]:
#Definiciones para antes de descargar los datos

#Crear lista para guardar los datos de sinac
lista_nacimientos <- list()

options(timeout = 600) #Aumentar el tiempo de espera para la descarga

In [14]:
#iterar por los años de descarga
    for(year in 19:23) {
#Crear la cadena de los años 2020 a 2023
        
        year_str <- paste0('20', year)
        #Cosntruir la url de descarga
        url <- paste0('http://www.dgis.salud.gob.mx/descargas/datosabiertos/nacimientos/sinac_', year_str, '.zip')
        
        #nombre y carpeta del archivo zip
        zip_file <- paste0('SINAC/sinac_', year_str, '.zip')
        extract_dir <- paste0('SINAC/sinac_', year_str)
#Crear carpeta si no existe
        if(!dir.exists('SINAC')) dir.create('SINAC')

#Intentar descargar archivos con trycatch
        tryCatch({
            download(url, destfile = zip_file, mode = 'wb')
            
            #Crear carpeta de extracción si no existe
            if(!file.exists(extract_dir)) dir.create(extract_dir)
                          
            #Descrompimir el archivo zip
            unzip(zip_file, exdir = extract_dir)

             #Buscar los archivos descomprimidos
            files <- list.files(extract_dir, full.names = TRUE)
    
    #Leer el primer archivo CSV encontrado (ajusta si es otro formato)
            csv_file <- files[grep("\\.csv$", files)][1]
    
        if (!is.na(csv_file)) {
        df <- read.csv(csv_file, stringsAsFactors = FALSE) %>% clean_names()
      
        # Guardar el dataframe en la lista
        lista_nacimientos[[year_str]] <- df
        
        # Eliminar el archivo ZIP
        file.remove(zip_file)
        
        message(paste('Año', year_str, 'procesado correctamente.'))
        } else {
        message(paste('No se encontró archivo CSV en', extract_dir))
        }
    }, error = function(e) {
        message(paste('Error al procesar el año', year_str))
    })
    }


: 

In [9]:
#Guardar la lista completa para no volver a descargar los datos
saveRDS(lista_nacimientos, file = 'SINAC/sinac_data.rds')

Descargar los datos del Censo Población y Vivienda 2020

In [10]:
#Carpeta donde se guardará todo
base_dir <- 'Censo Poblacion y Vivienda 2020'

#Url de descarga
url <- url <- 'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CPV_CB_Personas_ejemplo_csv.zip'

#Archivo zip y carpeta de extracción
extract_dir <- file.path(base_dir, 'INEGI. Censo 2020')

zip_file <- file.path(base_dir, 'pob_20.zip')

In [11]:
# Crear carpeta base si no existe
if (!dir.exists(base_dir)) dir.create(base_dir)

# Descargar ZIP (solo si no existe)
if (!file.exists(zip_file)) {
  download.file(url, destfile = zip_file, mode = 'wb')
}

# Descomprimir directamente en la carpeta con el nombre que deseas
if (!dir.exists(extract_dir)) {
  unzip(zip_file, exdir = extract_dir)
}

# Listar todos los archivos descomprimidos para verificar
files_all <- list.files(extract_dir, full.names = TRUE, recursive = TRUE)
print('Archivos descomprimidos:')
print(files_all)

# Buscar archivo CSV (sin importar mayúsculas/minúsculas)
files_csv <- list.files(extract_dir, pattern = "\\.[cC][sS][vV]$", full.names = TRUE, recursive = TRUE)

if (length(files_csv) > 0) {
  csv_file <- files_csv[1]  #Tomamos el único CSV que está
  pob_censo <- read.csv(csv_file, stringsAsFactors = FALSE) %>%
  clean_names()
  
  message(paste('Censo 2020 leído correctamente desde:', csv_file))
  
  # Guardar el dataframe para no volver a descargarlo
  saveRDS(pob_censo, file = 'Censo Poblacion y Vivienda 2020/censo_data.rds')
  
} else {
  message('No se encontró ningún archivo CSV en la carpeta descomprimida.')
}


[1] "Archivos descomprimidos:"
[1] "Censo Poblacion y Vivienda 2020/INEGI. Censo 2020/Censo2020_CPV_CB_Personas_ejemplo_csv.CSV"


Censo 2020 leído correctamente desde: Censo Poblacion y Vivienda 2020/INEGI. Censo 2020/Censo2020_CPV_CB_Personas_ejemplo_csv.CSV



Datos de población de CONAPO

In [12]:
#Carpeta donde se guardará todo
base_dir <- 'CONAPO'

#Url de descarga
url <- 'https://conapo.segob.gob.mx/work/models/CONAPO/pry23/DB/ConDem50a19_ProyPob20a70.zip'

#Archivo zip y carpeta de extracción
zip_file <- file.path(base_dir, 'conapo_data.zip')

extract_dir <- file.path(base_dir, 'ConDem50a19_ProyPob20a70')

In [ ]:
# Crear carpeta base si no existe
if (!dir.exists(base_dir)) dir.create(base_dir)

# Descargar el ZIP solo si no existe
if (!file.exists(zip_file)) {
  download.file(url, destfile = zip_file, mode = 'wb')
}

# Descomprimir solo si no se ha hecho antes
if (!dir.exists(extract_dir)) {
  unzip(zip_file, exdir = base_dir)
}

# Buscar archivo Excel dentro de la carpeta descomprimida
files <- list.files(extract_dir, pattern = "\\.xlsx$", full.names = TRUE)
excel_file <- files[1]  # Primer archivo Excel que encuentre

if (!is.na(excel_file)) {
  # Leer el Excel y limpiar nombres
  pob_conapo <- read_excel(excel_file) %>% clean_names()
  
  message('Archivo CONAPO leído correctamente.')

  # Guardar para no volver a descargar
  saveRDS(pob_conapo, file = 'CONAPO/poblacion_data.rds')
  
} else {
  message('No se encontró archivo Excel dentro de la carpeta descomprimida.')
}


Archivo CONAPO leído correctamente.



Cargar los datos descargados

In [3]:
#Función para cargar todos los datos descargados
datos_salud <- function(){
  list(
    defunciones = readRDS('Estadísticas de Defunciones Registradas (EDR)/edr_data.rds'),
    nacimientos = readRDS('SINAC/sinac_data.rds'),
    pob_censo = readRDS('Censo Poblacion y Vivienda 2020/censo_data.rds'),
    pob_conapo = readRDS('CONAPO/poblacion_data.rds')
  )
}

# Cargar los datos en un objeto
salud <- datos_salud()


Definimos los años para los cálculos

In [4]:
anios <- 2019:2023

Tasa de Mortalidad en menores de 5 años de la Ciudad de México

In [5]:
#Conjunto de nombres para los dataframes de defunciones
nombres_def <- c('2020' = 'defun20','2021' = 'defun21', '2022' = 'DEFUN22', '2023' = 'DEFUN23')

In [12]:
#Crear dataframe vacío para almacenar los resultados
natalidad_infantil <- tibble(anio = integer(), nacimientos = integer(), defunciones = integer(), tasa = numeric())


In [7]:
# Iterar sobre los años y calcular tasas
for (a in anios) {
  anio_str <- as.character(a)
  
  # Validar que existan datos para el año actual
  if (!is.null(salud$nacimientos[[anio_str]]) && !is.null(salud$defunciones[[anio_str]])) {
    
    # Extraer el dataframe de nacimientos
    df_nac <- salud$nacimientos[[anio_str]]
    
    # Obtener el nombre correcto del dataframe de defunciones
    nombre_df <- nombres_def[anio_str]
    df_def <- salud$defunciones[[anio_str]][[nombre_df]]
    
    # Calcular nacimientos para CDMX
    nac <- df_nac %>%
      filter(entidadresidencia == 9) %>%
      summarise(nacimientos = n()) %>%
      pull(nacimientos)
    
    # Calcular defunciones infantiles para CDMX
    def <- df_def %>%
      filter(ent_resid == '09', anio_ocur == a, edad >= 1001, edad < 4005) %>%
      summarise(defunciones = n()) %>%
      pull(defunciones)
    
    # Calcular la tasa
    tasa <- (def / nac) * 1000
    
    # Guardar resultados
    mortalidad_infantil <- mortalidad_infantil %>% 
      add_row(anio = a, nacimientos = nac, defunciones = def, tasa = tasa)
    
  } else {
    message(paste('Datos incompletos para el año', a))
  }
}

# Mostrar resultados
print(mortalidad_infantil)


Datos incompletos para el año 2019



# A tibble: 4 × 4
   anio nacimientos defunciones  tasa
  <int>       <int>       <int> <dbl>
1  2020       87005        1266  14.6
2  2021       77476        1062  13.7
3  2022       80892        1159  14.3
4  2023       78032        1152  14.8


Total de nacimientos ocurridos en niñas y adolescentes menores de 15 años 2019-2023

*Secretaría de Salud utiliza la variable Entidad de federativa del parto/certificado*

In [8]:
#Crear tabla para guiardar los  resultados
madres_menores_15 <- tibble(anio = integer(), nacimientos = integer())

In [9]:
for (a in anios) {
    anio_str <- as.character(a)

    if (!is.null(salud$nacimientos[[anio_str]])) {
        df_nac <- salud$nacimientos[[anio_str]] %>% as_tibble()
        
        # Definir las variables Entidad y Edad por año
        if (a == 2019) {
            var_entidad <- 'ent_cert'
            var_edad <- 'edadm'
        } else {
            var_entidad <- 'entidadfederativacertifica'
            var_edad <- 'edad'
        }
        
        # Filtrar y calcular
        nacimientos_15 <- df_nac %>%
            filter(.data[[var_entidad]] == 9, .data[[var_edad]] < 15) %>%
            summarise(nacimientos = n()) %>%
            pull(nacimientos)
        
        # Guardar los resultados en la tabla creada
        madres_menores_15 <- madres_menores_15 %>%
            add_row(anio = a, nacimientos = nacimientos_15)
        
    } else {
        message(paste('Datos incompletos para el año', a))
    }
}

# Ver resultados
print(madres_menores_15)

# A tibble: 5 × 2
   anio nacimientos
  <int>       <int>
1  2019         322
2  2020         272
3  2021         321
4  2022         304
5  2023         299


Mortalidad por enfermedades del corazón

*De acuerdo con INEGI, se clasifica la mortalidad por enfermedades del corazón (con excepción del paro cardiaco) utilizando el conjunto de códigos de la Lista Mexicana de Enfermedades del 26 al 29.*

In [5]:
#Claves enfermedades del corazón para filtrarlos en cada año
enfer_cora <- c('26', '26A', '26B', '27', '27A', '27B', '27Z', '28', '28A', '28Z', '29', '29A', '29B', '29C', '29D', '29Z')

In [6]:
tasa_mortalidad_corazon <- list()

In [8]:
for (a in anios) {
  anio_str <- as.character(a)

  if (!is.null(salud$defunciones[[anio_str]])) {

    # Buscar el nombre de la tabla de defunciones de ese año
    nombres_tablas <- names(salud$defunciones[[anio_str]])
    tabla_defun <- nombres_tablas[str_detect(nombres_tablas, regex('defun', ignore_case = TRUE))][1]

    # Cargar la tabla como tibble
    df_def <- salud$defunciones[[anio_str]][[tabla_defun]] %>% as_tibble()

    # Filtrar por los códigos de enfermedades del corazón usando directamente lista_mex
    df_filtrado <- df_def %>%
      filter(lista_mex %in% enfer_cora, ent_resid == '09') %>%
      select(ent_resid, lista_mex) %>%
      summarise(muertes = n()) %>%
      pull(muertes)

    # Guardar el resultado en la lista
    tasa_mortalidad_corazon[[anio_str]] <- df_filtrado

    message(paste('Año', a, 'procesado correctamente. Registros filtrados:', nrow(df_filtrado)))

  } else {
    message(paste('Datos incompletos para el año', a))
  }
}


Datos incompletos para el año 2019

Año 2020 procesado correctamente. Registros filtrados: 

Año 2021 procesado correctamente. Registros filtrados: 

Año 2022 procesado correctamente. Registros filtrados: 

Año 2023 procesado correctamente. Registros filtrados: 



In [9]:
print(tasa_mortalidad_corazon)

$`2020`
[1] 21064

$`2021`
[1] 18610

$`2022`
[1] 16810

$`2023`
[1] 17020



In [10]:
pob_censo <- tibble(salud$pob_censo)

In [ ]:
#Población censo
pob_censo <- pob_censo %>%
filter(ent =='09') %>%
summarise(poblacion_censo)

In [ ]:
# 1. Población por año para CDMX
# Suponiendo que pob_censo y pob_conapo tienen columnas: anio, ent, poblacion

# Obtener población 2020 del censo
pob_cdmx_2020 <- salud$pob_censo %>%
  filter(ent == 9) %>%
  summarise(poblacion = n()) %>%
  pull(poblacion)


# Crear tabla con muertes
tasa_mortalidad_corazon_df <- tibble(
  anio = as.integer(names(tasa_mortalidad_corazon)),
  muertes = unlist(tasa_mortalidad_corazon)
)

# Añadir población
tasa_mortalidad_corazon_df <- tasa_mortalidad_corazon_df %>%
  mutate(
    poblacion = case_when(
      anio == 2020 ~ pob_cdmx_2020,
      TRUE ~ pob_cdmx_conapo$poblacion[match(anio, pob_cdmx_conapo$anio)]
    ),
    tasa = (muertes / poblacion) * 100000
  )

# Resultado final
print(tasa_mortalidad_corazon_df)


In [ ]:
# Obtener población 2021-2023 del CONAPO
pob_cdmx_conapo <- salud$pob_conapo %>%
  filter(ent == 9, anio %in% c(2021, 2022, 2023)) %>%
  group_by(anio) %>%
  summarise(poblacion = n()) %>%
  ungroup()

